<a href="https://colab.research.google.com/github/HibaAp/RAG-KnowledgeBase-System/blob/main/DailyUpdates/20_02_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install langchain langchain-community pdfplumber numpy scikit-learn faiss-cpu requests langchain-groq googlesearch-python beautifulsoup4 langchain-experimental sentence_transformers

  Using cached langchain_community-0.3.18-py3-none-any.whl.metadata (2.4 kB)
  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
  Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
  Using cached langchain_groq-0.2.4-py3-none-any.whl.metadata (3.0 kB)
  Using cached googlesearch_python-1.3.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_core-0.3.37-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain-0.3.19-py3-none-any.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (48 kB)
  Using cached

In [1]:
groq_api_key = "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"


In [5]:
from typing import List
from bs4 import BeautifulSoup
import requests
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.hyde.base import HypotheticalDocumentEmbedder
from langchain_groq import ChatGroq



def web_search(query, max_results=3):
    """Perform web search using googlesearch-python"""
    from googlesearch import search
    results = list(search(query, num_results=max_results))
    return results[:max_results]

def fetch_content_from_link(link):
    try:
        if not link.startswith(('http://', 'https://')):
            link = f'https://{link}'
        response = requests.get(link, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        raw_text = soup.get_text()
        cleaned_text = ' '.join(raw_text.split())
        return cleaned_text
    except Exception as e:
        #print(f"Error fetching {link}: {str(e)}")
        return ""


# Monkey-patch with proper input_variables access
@property
def fixed_input_keys(self) -> List[str]:
    return self.llm_chain.prompt.input_variables  # Access through prompt

HypotheticalDocumentEmbedder.input_keys = fixed_input_keys

def get_retrievers(pdf_path, groq_api_key):
    import warnings
    warnings.filterwarnings("ignore")
    import random
    import pdfplumber
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity
    from langchain.embeddings import HuggingFaceBgeEmbeddings
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_community.document_loaders import PyPDFLoader
    from langchain.docstore.document import Document
    from langchain_community.vectorstores import FAISS

    # Initialize HyDE components
    hyde_prompt = PromptTemplate(
        input_variables=["question","web_con"],
        template="""Generate a comprehensive hypothetical answer to: {question} based on the following web content: {web_con}.
    Include key facts, concepts, and relevant context."""
    )

    hyde_llm = ChatGroq(
        groq_api_key=groq_api_key,
        model_name="llama3-70b-8192",
        temperature=0.1
    )
    hyde_chain = LLMChain(llm=hyde_llm, prompt=hyde_prompt)

    # Base embeddings model
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-large-en",
        encode_kwargs={'normalize_embeddings': False}
    )

    # Wrap with HyDE (using patched version)
    hyde_embeddings = HypotheticalDocumentEmbedder(
        llm_chain=hyde_chain,
        base_embeddings=embedding_model,
    )
    def embed_texts(texts):
        return embedding_model.embed_documents(texts)


    def get_header_footer(pdf_path, threshold=0.71):
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            if total_pages >= 15:
                random_page_nos = random.sample(range(5, total_pages), 10)
            else:
                random_page_nos = list(range(total_pages))

            avg_similarity = 1
            header_lines = -1
            while avg_similarity > threshold and header_lines < 4:
                header_lines += 1
                five_lines = []
                for page_no in random_page_nos:
                    lines = pdf.pages[page_no].extract_text().split('\n')
                    if len(lines) > header_lines:
                        five_lines.append(lines[header_lines])
                similarities = cosine_similarity(embed_texts(five_lines))
                avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])

            avg_similarity = 1
            footer_lines = -1
            while avg_similarity > threshold and footer_lines < 4:
                footer_lines += 1
                five_lines = []
                for page_no in random_page_nos:
                    lines = pdf.pages[page_no].extract_text().split('\n')
                    if len(lines) > footer_lines:
                        five_lines.append(lines[-(footer_lines + 1)])
                similarities = cosine_similarity(embed_texts(five_lines))
                avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])
            return header_lines, footer_lines


    def extract_text(pdf_path):
        header_lines, footer_lines = get_header_footer(pdf_path)
        with pdfplumber.open(pdf_path) as pdf:
            text = ''
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    lines = page_text.split('\n')
                    if lines:
                        page_text = '\n'.join(lines[header_lines:-(footer_lines + 1)])
                        text += page_text + '\n'
            return text

    text = extract_text(pdf_path)

    def get_vectorstore1():
        texts = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_text(text)
        docs = [Document(text) for text in texts if text.strip()]
        vectorstore = FAISS.from_documents(docs, hyde_embeddings)  # Use HyDE embeddings
        return vectorstore

    def get_vectorstore2():
        texts = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=400).split_text(text)
        docs = [Document(text) for text in texts if text.strip()]
        vectorstore = FAISS.from_documents(docs, hyde_embeddings)  # Use HyDE embeddings
        return vectorstore

    retriever1 = get_vectorstore1().as_retriever(search_kwargs={"k": 6})
    retriever2 = get_vectorstore2().as_retriever(search_kwargs={"k": 3})
    return retriever1, retriever2




def get_answer(query, retriever1, retriever2, groq_api_key):
    # Web search integration
    links = web_search(query)
    web_results = "\n".join([f"{i+1}. {fetch_content_from_link(link)}" for i, link in enumerate(links)])

    # HyDE-enhanced document retrieval
    doc_results1 = retriever1.get_relevant_documents(query)
    doc_results2 = retriever2.get_relevant_documents(query)
    doc_context = "\n---\n".join([doc.page_content for doc in doc_results1 + doc_results2])

    # Context management
    combined_context = f"""
    WEB SEARCH RESULTS:
    {web_results}

    DOCUMENT CONTENT:
    {doc_context}
    """
    if len(combined_context) > 4000:
        combined_context = combined_context[:4000]

    # LLM initialization
    llm = ChatGroq(
        api_key=groq_api_key,  # Ensuring correct argument name
        model_name="llama3-70b-8192",
        temperature=0.05
    )

    # Corrected prompt template
    prompt_template = """
    Analyze and synthesize information from both web results and document content to answer
    the question. Follow these steps:
    1. Identify key facts from web results.
    2. Find supporting information in documents.
    3. Combine insights from both sources.
    4. If sources conflict, note this and prioritize document content.
    5. Provide a clear, concise answer.
    6. Do not restate the question. Provide a direct comparison of the answers.
    7. Give a final judgment on which answer is better and why, without using phrases like 'based on web results' or unnecessary explanations.

    CONTEXT:
    {combined_context}

    QUESTION: {question}

    FINAL ANSWER:
    """
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["combined_context", "question"]  # Fixed placeholder reference
    )

    chain = LLMChain(llm=llm, prompt=prompt)

    return chain.run(combined_context=combined_context, question=query)  # Matching argument names


def compare_answers(query, retriever1, retriever2, retriever3, retriever4, groq_api_key):
    answer1 = get_answer(query, retriever1, retriever2, groq_api_key)
    answer2 = get_answer(query, retriever3, retriever4, groq_api_key)

    comparison_prompt = f"""
    Compare the two answers given for the same question:

    QUESTION: {query}

    ANSWER 1: {answer1}

    ANSWER 2: {answer2}

    Do not restate the question. Provide a direct comparison of the answers focusing only on:
    1. Factual consistency
    2. Source reliability
    3. Completeness of information
    4. Clarity of presentation

    Give a final judgment on which answer is better and why, without using phrases like 'based on web results' or unnecessary explanations.
    """


    llm = ChatGroq(
        groq_api_key=groq_api_key,
        model_name="llama3-70b-8192",
        temperature=0.05
    )

    return llm.invoke(comparison_prompt).content

In [17]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.hyde.base import HypotheticalDocumentEmbedder
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import pdfplumber
import random
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
def web_search(query, max_results=3):
  """Perform web search using googlesearch-python"""
  from googlesearch import search
  results = list(search(query, num_results=max_results))
  return results[:max_results]

def fetch_content_from_link(link):
    try:
        if not link.startswith(('http://', 'https://')):
            link = f'https://{link}'
        response = requests.get(link, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        raw_text = soup.get_text()
        cleaned_text = ' '.join(raw_text.split())
        return cleaned_text
    except Exception as e:
        #print(f"Error fetching {link}: {str(e)}")
        return ""

def get_retrievers(pdf_path,groq_api_key):

    hyde_prompt = PromptTemplate(
        input_variables=["question"],
        template="""Generate a comprehensive hypothetical answer to: {question}
    Include key facts, concepts, and relevant context."""
    )

    hyde_llm = ChatGroq(
        groq_api_key=groq_api_key,
        model_name="llama3-70b-8192",
        temperature=0.1
    )
    hyde_chain = LLMChain(llm=hyde_llm, prompt=hyde_prompt)

    # Base embeddings model
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-large-en",
        encode_kwargs={'normalize_embeddings': False}
    )

    # HyDE wrapper
    hyde_embeddings = HypotheticalDocumentEmbedder(
        llm_chain=hyde_chain,
        base_embeddings=embedding_model,
    )

    def embed_texts(texts):
        return embedding_model.embed_documents(texts)

    def get_header_footer(pdf_path, threshold=0.71):
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            if total_pages >= 15:
                random_page_nos = random.sample(range(5, total_pages), 10)
            else:
                random_page_nos = list(range(total_pages))

            avg_similarity = 1
            header_lines = -1
            while avg_similarity > threshold and header_lines < 4:
                header_lines += 1
                five_lines = []
                for page_no in random_page_nos:
                    lines = pdf.pages[page_no].extract_text().split('\n')
                    if len(lines) > header_lines:
                        five_lines.append(lines[header_lines])
                similarities = cosine_similarity(embed_texts(five_lines))
                avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])

            avg_similarity = 1
            footer_lines = -1
            while avg_similarity > threshold and footer_lines < 4:
                footer_lines += 1
                five_lines = []
                for page_no in random_page_nos:
                    lines = pdf.pages[page_no].extract_text().split('\n')
                    if len(lines) > footer_lines:
                        five_lines.append(lines[-(footer_lines + 1)])
                similarities = cosine_similarity(embed_texts(five_lines))
                avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])
            return header_lines, footer_lines

    def extract_text(pdf_path):
        header_lines, footer_lines = get_header_footer(pdf_path)
        with pdfplumber.open(pdf_path) as pdf:
            text = ''
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    lines = page_text.split('\n')
                    if lines:
                        page_text = '\n'.join(lines[header_lines:-(footer_lines + 1)])
                        text += page_text + '\n'
            return text

    text = extract_text(pdf_path)
    texts = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_text(text)
    docs = [Document(page_content=t) for t in texts if t.strip()]

    # FAISS index using base embeddings
    vectorstore = FAISS.from_documents(docs, embedding_model)


    class WebEnhancedHydeRetriever:
    def __init__(self, vectorstore, embedding_model, hyde_llm):
        self.vectorstore = vectorstore
        self.embedding_model = embedding_model
        self.hyde_llm = hyde_llm  # HydeLLM model for contextual query generation

    def get_relevant_documents(self, query, k=5):
        """Retrieve documents using web search, HydeLLM refinement, and similarity search."""

        def get_web_results(query):
            """Perform a web search and fetch page content."""
            try:
                links = web_search(query)  # Function to search the web
                return [fetch_content_from_link(link) for link in links]
            except Exception as e:
                print(f"Web search error: {e}")
                return []

        # Step 1: Web search and content retrieval
        web_results = get_web_results(query)

        # Step 2: If no web results, fallback to direct vectorstore search
        if not web_results:
            return self.vectorstore.similarity_search(query, k=k)

        # Step 3: Pass the web content + query to HydeLLM for refinement
        combined_web_text = "\n".join(web_results)
        # Fix: Create a HumanMessage object for the query
        from langchain import HumanMessage
        refined_query = self.hyde_llm.generate([HumanMessage(content=query, additional_kwargs={"context": combined_web_text})]).generations[0][0].text

        # Step 4: Embed the refined query and perform similarity search
        refined_query_embedding = self.embedding_model.embed_query(refined_query)
        return self.vectorstore.similarity_search_by_vector(refined_query_embedding, k=k)
    return  WebEnhancedHydeRetriever(vectorstore, embedding_model, hyde_llm)
def get_answer(query, retriever1, groq_api_key):
    from langchain.prompts import PromptTemplate
    from langchain_groq import ChatGroq
    from langchain.chains import LLMChain

    # 2. Retrieve document content
    doc_results1 = retriever1.get_relevant_documents(query)
    #doc_results2 = retriever2.get_relevant_documents(query)
    doc_context = "\n---\n".join([doc.page_content for doc in doc_results1 ])



    # 4. Create LLM chain with combined context
    llm = ChatGroq(groq_api_key=groq_api_key, model='llama3-70b-8192', temperature=0.05)

    prompt_template = """
    Your task is to answer the question, using only the information provided in the given context.
    The answer should be accuate and detailed.
    Where applicable, refer to specific section numbers within the context (e.g., "According to section 4.1.2,...").
    If the answer is not found in the provided context, simply state that there is no relevant information available without sharing details about the context.

    CONTEXT: {context}

    QUESTION: {question}

    FINAL ANSWER:
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(context=doc_context, question=query)

In [9]:
pdf_path="/content/ABSAReferences (1).pdf"

In [13]:
retriever1=get_retrievers(pdf_path,groq_api_key)

In [ ]:
retriever1

In [8]:
query="list me some absa related papers"

In [18]:
get_answer(query,retriever1,groq_api_key)

TypeError: Got unknown type l

In [22]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.hyde.base import HypotheticalDocumentEmbedder
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.schema import HumanMessage, SystemMessage
import pdfplumber
import random
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
from bs4 import BeautifulSoup


def web_search(query, max_results=3):
    """Perform web search using googlesearch-python"""
    from googlesearch import search
    try:
        results = list(search(query, num_results=max_results))
        return results[:max_results]
    except Exception as e:
        print(f"Web search error: {e}")
        return []


def fetch_content_from_link(link):
    try:
        if not link.startswith(('http://', 'https://')):
            link = f'https://{link}'
        response = requests.get(link, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        raw_text = soup.get_text()
        cleaned_text = ' '.join(raw_text.split())
        return cleaned_text
    except Exception as e:
        print(f"Error fetching {link}: {str(e)}")
        return ""


def get_retrievers(pdf_path, groq_api_key):
    # Modified Hyde prompt to include web context
    hyde_prompt = PromptTemplate(
        input_variables=["question", "context"],
        template="""Using the following web search context, generate a comprehensive hypothetical answer to the question.
        Include key facts, concepts, and relevant context from both the question and provided information.

        WEB CONTEXT: {context}
        QUESTION: {question}

        HYPOTHETICAL ANSWER:"""
    )

    hyde_llm = ChatGroq(
        groq_api_key=groq_api_key,
        model_name="llama3-70b-8192",
        temperature=0.1
    )
    hyde_chain = LLMChain(llm=hyde_llm, prompt=hyde_prompt)

    # Base embeddings model
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-large-en",
        encode_kwargs={'normalize_embeddings': False}
    )

    # HyDE wrapper
    hyde_embeddings = HypotheticalDocumentEmbedder(
        llm_chain=hyde_chain,
        base_embeddings=embedding_model,
    )

    def embed_texts(texts):
        return embedding_model.embed_documents(texts)

    def get_header_footer(pdf_path, threshold=0.71):
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            if total_pages >= 15:
                random_page_nos = random.sample(range(5, total_pages), 10)
            else:
                random_page_nos = list(range(total_pages))

            avg_similarity = 1
            header_lines = -1
            while avg_similarity > threshold and header_lines < 4:
                header_lines += 1
                five_lines = []
                for page_no in random_page_nos:
                    lines = pdf.pages[page_no].extract_text().split('\n')
                    if len(lines) > header_lines:
                        five_lines.append(lines[header_lines])
                similarities = cosine_similarity(embed_texts(five_lines))
                avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])

            avg_similarity = 1
            footer_lines = -1
            while avg_similarity > threshold and footer_lines < 4:
                footer_lines += 1
                five_lines = []
                for page_no in random_page_nos:
                    lines = pdf.pages[page_no].extract_text().split('\n')
                    if len(lines) > footer_lines:
                        five_lines.append(lines[-(footer_lines + 1)])
                similarities = cosine_similarity(embed_texts(five_lines))
                avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])
            return header_lines, footer_lines

    def extract_text(pdf_path):
        header_lines, footer_lines = get_header_footer(pdf_path)
        with pdfplumber.open(pdf_path) as pdf:
            text = ''
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    lines = page_text.split('\n')
                    if lines:
                        page_text = '\n'.join(lines[header_lines:-(footer_lines + 1)])
                        text += page_text + '\n'
            return text

    text = extract_text(pdf_path)
    texts = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_text(text)
    docs = [Document(page_content=t) for t in texts if t.strip()]

    # FAISS index using base embeddings
    vectorstore = FAISS.from_documents(docs, embedding_model)



    class WebEnhancedHydeRetriever:
        def __init__(self, vectorstore, embedding_model, hyde_chain):
            self.vectorstore = vectorstore
            self.embedding_model = embedding_model
            self.hyde_chain = hyde_chain

        def get_relevant_documents(self, query, k=5):
            """Retrieve documents using web-enhanced Hyde refinement."""
            links = web_search(query)
            web_results = "\n".join([f"{i+1}. {fetch_content_from_link(link)}" for i, link in enumerate(links)])

            if not web_results:
                return self.vectorstore.similarity_search(query, k=k)

            combined_web_text = "\n".join(web_results)[:3000]  # Truncate to prevent token overflow

            # Generate hypothetical answer using BOTH web context and original query
            refined_query = self.hyde_chain.invoke({
                "question": query,
                "context": combined_web_text
            })["text"]

            # Embed and search
            query_embedding = self.embedding_model.embed_query(refined_query)
            return self.vectorstore.similarity_search_by_vector(query_embedding, k=k)



    return WebEnhancedHydeRetriever(vectorstore, embedding_model, hyde_chain)


def get_answer(query, retriever1, groq_api_key):
    from langchain.prompts import PromptTemplate
    from langchain_groq import ChatGroq
    from langchain.chains import LLMChain


    doc_results = retriever1.get_relevant_documents(query)
    doc_context = "\n---\n".join([doc.page_content for doc in doc_results if doc.page_content.strip()])

    llm = ChatGroq(groq_api_key=groq_api_key, model='llama3-70b-8192', temperature=0.05)

    llm = ChatGroq(groq_api_key=groq_api_key, model='llama3-70b-8192', temperature=0.05)

    prompt_template = """
    you are intelligent chatbot who will answer the legal document related queries.
    Your task is to answer the question, using only the information provided in the given context.
    The answer should be accuate and detailed.
    Where applicable, refer to specific section numbers within the context (e.g., "According to section 4.1.2,...").
    If the answer is not found in the provided context, simply state that there is no relevant information available without sharing details about the context.
    just give the answer only, avoid tems like "based on provides context..."


    CONTEXT: {context}

    QUESTION: {question}

    FINAL ANSWER:
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(context=doc_context, question=query)



In [3]:
pdf_path="/content/ABSAReferences (1).pdf"

In [5]:
groq_api_key = "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"


In [20]:
retriever1=get_retrievers(pdf_path,groq_api_key)

In [14]:
query="list me some absa related papers"

In [23]:
get_answer(query,retriever1, groq_api_key)

'Here are some ABSA related papers mentioned in the context:\n\n1. https://www.mdpi.com/2079-9292/10/21/2641\n2. https://thesai.org/Downloads/Volume13No12/Paper_112-Aspect_based_Sentiment_Analysis_for_Bengali_Text.pdf\n3. https://aclanthology.org/2025.coling-main.391.pdf\n4. https://www.cse.iitb.ac.in/~pb/papers/lrec16-sentiment-resource.pdf\n5. https://www.cfilt.iitb.ac.in/resources/surveys/2022/kunal_CrossLingualABSA_survey_2022.pdf\n6. https://aclanthology.org/2020.lrec-1.617/\n7. https://iitp.ac.in/~shad.pcs15/data/Tutorial-SA-Hindi-GIAN.pdf\n8. https://www.cse.iitb.ac.in/~pb/papers/cicling16-aspect-based-sa.pdf\n9. https://aclanthology.org/L16-1429/\n10. https://www.researchgate.net/publication/372289987_ASPECT-BASED_SENTIMENT_ANALYSIS_A_COMPREHENSIVE_SURVEY_OF_TECHNIQUES_AND_APPLICATIONS\n17. https://ieeexplore.ieee.org/document/9402365/\n18. https://dl.acm.org/doi/10.1145/3485243\n19. https://www.researchgate.net/publication/379119554_Rule-Based_Approach_in_Aspect-Based_Sentimen